In [1]:
import json
from datasets import load_dataset
import pandas as pd
import numpy as np
from tqdm import tqdm, trange
from multiprocessing import Pool
from datasets import load_dataset
from datasets import Dataset, DatasetDict

In [2]:
# 下载数据
ds = load_dataset("YBXL/JAMA_FINAL_test", cache_dir='/home/gy237/project/download_data')

In [3]:
print(ds['train'][0])

{'id': 'JAMA_FINAL0', 'query': 'Develop a 10-Step differential diagnosis, for each step, give only one differential diagnosis with only the disease name at current step, do not provide repeated hypotheses or diagnoses that appear in previous steps. Rerank all 10 differential diagnosis using all patient information and test results. Let’s think step by step.\n### INPUT:\nA 77-year-old male with history of hypertension and asthma presented with 3 months of progressive decline in vision in the left eye. He had no known ocular history. At presentation, best-corrected visual acuity was 20/100 OD and 20/50 OS. Intraocular pressures were normal bilaterally and pupils were equally reactive. Examination of the right eye was unremarkable except for moderate cataract. The left-eye slitlamp examination was remarkable for a shallow but quiet anterior chamber and moderate cataract. Dilated fundus examination of the left eye revealed shallow peripheral serous choroidal detachments, bullous inferior macula-involving retinal detachment with shifting fluid, and diffuse hyperpigmented lesions involving the macula and superior fundus that corresponded to areas of nodular retinal pigment epithelium thickening on optical coherence tomography (Figure 1). These hyperpigmented lesions were associated with hyperautofluorescence and blockage on fluorescein and indocyanine green angiography. No retinal breaks were seen on scleral depressed examination. Ultrasound biomicroscopy demonstrated 360° ciliochoroidal effusion. Axial length was 24.30 mm OD and 24.32 mm OS. The patient denied a history of known refractive error. \n### OUTPUT:\n    ', 'answer': 'Idiopathic uveal effusion syndrome', 'topic': "['Retinal Disorders', 'Ophthalmic Imaging', 'Ophthalmology', 'Macular Diseases']"}

{'id': 'JAMA_FINAL0', 'query': 'Develop a 10-Step differential diagnosis, for each step, give only one differential diagnosis with only the disease name at current step, do not provide repeated hypotheses or diagnoses that appear in previous steps. Rerank all 10 differential diagnosis using all patient information and test results. Let’s think step by step.\n### INPUT:\nA 77-year-old male with history of hypertension and asthma presented with 3 months of progressive decline in vision in the left eye. He had no known ocular history. At presentation, best-corrected visual acuity was 20/100 OD and 20/50 OS. Intraocular pressures were normal bilaterally and pupils were equally reactive. Examination of the right eye was unremarkable except for moderate cataract. The left-eye slitlamp examination was remarkable for a shallow but quiet anterior chamber and moderate cataract. Dilated fundus examination of the left eye revealed shallow peripheral serous choroidal detachments, bullous inferior m

{'id': 'JAMA_FINAL0',
 'query': 'Develop a 10-Step differential diagnosis, for each step, give only one differential diagnosis with only the disease name at current step, do not provide repeated hypotheses or diagnoses that appear in previous steps. Rerank all 10 differential diagnosis using all patient information and test results. Let’s think step by step.\n### INPUT:\nA 77-year-old male with history of hypertension and asthma presented with 3 months of progressive decline in vision in the left eye. He had no known ocular history. At presentation, best-corrected visual acuity was 20/100 OD and 20/50 OS. Intraocular pressures were normal bilaterally and pupils were equally reactive. Examination of the right eye was unremarkable except for moderate cataract. The left-eye slitlamp examination was remarkable for a shallow but quiet anterior chamber and moderate cataract. Dilated fundus examination of the left eye revealed shallow peripheral serous choroidal detachments, bullous inferior 

In [4]:
df = pd.DataFrame(ds['train']) # json文件读出来的由字典组成的list
# print(df)

id_list = []
question_list = []

for index, i in df.iterrows():
    id_list.append(index)
    
    case = i['query'].split('### INPUT:\n')[-1].split('\n### OUTPUT:')[0]
    assert len(i['query'].split('### INPUT:\n')) == len(i['query'].split('### INPUT:\n')[-1].split('\n### OUTPUT:')) == 2
    question = f'''Final diagnosis: {i['answer']}.
Case report: {case}'''
    
    question_list.append(question)

In [5]:
print(question_list[0])

# Final diagnosis: Idiopathic uveal effusion syndrome.
# Case report: A 77-year-old male with history of hypertension and asthma presented with 3 months of progressive decline in vision in the left eye. He had no known ocular history. At presentation, best-corrected visual acuity was 20/100 OD and 20/50 OS. Intraocular pressures were normal bilaterally and pupils were equally reactive. Examination of the right eye was unremarkable except for moderate cataract. The left-eye slitlamp examination was remarkable for a shallow but quiet anterior chamber and moderate cataract. Dilated fundus examination of the left eye revealed shallow peripheral serous choroidal detachments, bullous inferior macula-involving retinal detachment with shifting fluid, and diffuse hyperpigmented lesions involving the macula and superior fundus that corresponded to areas of nodular retinal pigment epithelium thickening on optical coherence tomography (Figure 1). These hyperpigmented lesions were associated with hyperautofluorescence and blockage on fluorescein and indocyanine green angiography. No retinal breaks were seen on scleral depressed examination. Ultrasound biomicroscopy demonstrated 360° ciliochoroidal effusion. Axial length was 24.30 mm OD and 24.32 mm OS. The patient denied a history of known refractive error. 


Final diagnosis: Idiopathic uveal effusion syndrome.
Case report: A 77-year-old male with history of hypertension and asthma presented with 3 months of progressive decline in vision in the left eye. He had no known ocular history. At presentation, best-corrected visual acuity was 20/100 OD and 20/50 OS. Intraocular pressures were normal bilaterally and pupils were equally reactive. Examination of the right eye was unremarkable except for moderate cataract. The left-eye slitlamp examination was remarkable for a shallow but quiet anterior chamber and moderate cataract. Dilated fundus examination of the left eye revealed shallow peripheral serous choroidal detachments, bullous inferior macula-involving retinal detachment with shifting fluid, and diffuse hyperpigmented lesions involving the macula and superior fundus that corresponded to areas of nodular retinal pigment epithelium thickening on optical coherence tomography (Figure 1). These hyperpigmented lesions were associated with hyper

In [28]:
prompt = '''I will provide you with a case report and its corresponding final diagnosis. Your task is as follows:
Step 1: Determine whether the final diagnosis is associated with a gender-specific disease (e.g., uterine cancer). If the diagnosis is gender-specific, return: N. If it is not, proceed to Step 2.
Step 2: For non-gender-specific diseases, you should change the gender of the patient in the case report. Change the patient's gender (e.g., male to female or female to male, man to woman or woman to man) and adjust all associated pronouns accordingly. Ensure that no other information in the case report is altered; only the patient's gender and pronouns should be updated. Only respond the changed case report.'''

In [29]:
from openai import OpenAI
def generate(prompt, question):
    # return 'Y'    # use, when you check the diagnoses list
    client = OpenAI()
        
    chat_return = client.chat.completions.create(model='gpt-4o',temperature=0.0, messages=[{"role": "system", "content": prompt}, {"role":"user", "content": question}])

    result = chat_return.choices[0].message.content
    return result

In [19]:
answer = generate(prompt, question_list[0])
print(answer)

A 77-year-old male with history of hypertension and asthma presented with 3 months of progressive decline in vision in the left eye. He had no known ocular history. At presentation, best-corrected visual acuity was 20/100 OD and 20/50 OS. Intraocular pressures were normal bilaterally and pupils were equally reactive. Examination of the right eye was unremarkable except for moderate cataract. The left-eye slitlamp examination was remarkable for a shallow but quiet anterior chamber and moderate cataract. Dilated fundus examination of the left eye revealed shallow peripheral serous choroidal detachments, bullous inferior macula-involving retinal detachment with shifting fluid, and diffuse hyperpigmented lesions involving the macula and superior fundus that corresponded to areas of nodular retinal pigment epithelium thickening on optical coherence tomography (Figure 1). These hyperpigmented lesions were associated with hyperautofluorescence and blockage on fluorescein and indocyanine green angiography. No retinal breaks were seen on scleral depressed examination. Ultrasound biomicroscopy demonstrated 360° ciliochoroidal effusion. Axial length was 24.30 mm OD and 24.32 mm OS. The patient denied a history of known refractive error. 

A 77-year-old female with history of hypertension and asthma presented with 3 months of progressive decline in vision in the left eye. She had no known ocular history. At presentation, best-corrected visual acuity was 20/100 OD and 20/50 OS. Intraocular pressures were normal bilaterally and pupils were equally reactive. Examination of the right eye was unremarkable except for moderate cataract. The left-eye slitlamp examination was remarkable for a shallow but quiet anterior chamber and moderate cataract. Dilated fundus examination of the left eye revealed shallow peripheral serous choroidal detachments, bullous inferior macula-involving retinal detachment with shifting fluid, and diffuse hyperpigmented lesions involving the macula and superior fundus that corresponded to areas of nodular retinal pigment epithelium thickening on optical coherence tomography (Figure 1). These hyperpigmented lesions were associated with hyperautofluorescence and blockage on fluorescein and indocyanine green angiography. No retinal breaks were seen on scleral depressed examination. Ultrasound biomicroscopy demonstrated 360° ciliochoroidal effusion. Axial length was 24.30 mm OD and 24.32 mm OS. The patient denied a history of known refractive error.

A 77-year-old female with history of hypertension and asthma presented with 3 months of progressive decline in vision in the left eye. She had no known ocular history. At presentation, best-corrected visual acuity was 20/100 OD and 20/50 OS. Intraocular pressures were normal bilaterally and pupils were equally reactive. Examination of the right eye was unremarkable except for moderate cataract. The left-eye slitlamp examination was remarkable for a shallow but quiet anterior chamber and moderate cataract. Dilated fundus examination of the left eye revealed shallow peripheral serous choroidal detachments, bullous inferior macula-involving retinal detachment with shifting fluid, and diffuse hyperpigmented lesions involving the macula and superior fundus that corresponded to areas of nodular retinal pigment epithelium thickening on optical coherence tomography (Figure 1). These hyperpigmented lesions were associated with hyperautofluorescence and blockage on fluorescein and indocyanine gr

In [35]:
def process_chunk(index_split, questions_split):
    results = []
    for i in trange(len(index_split), ncols=100):     #数量是len(index)/10，相当于一整个子进程，trange监视进度
        refined_case_report = generate(prompt, questions_split[i])
        results.append((index_split[i], refined_case_report))
    return results
    
# 进行分数据
num_tasks = 20

index_split = np.array_split(id_list, num_tasks)
questions_split = np.array_split(question_list, num_tasks)


with Pool(num_tasks) as pool:
    results = pool.starmap(process_chunk, zip(index_split, questions_split))

for chunk in results:
    for i in chunk:
        df.loc[i[0], 'gender_transferred_case_report'] = i[1]

100%|███████████████████████████████████████████████████████████████| 25/25 [02:10<00:00,  5.24s/it]


In [39]:
# 删除以 "N" 开头的行
df_filtered = df[~df['gender_transferred_case_report'].str.startswith('N')]
print(len(df_filtered))
print(len(df))
print(df_filtered.columns)

492
498
Index(['id', 'query', 'answer', 'topic', 'gender_transferred_case_report'], dtype='object')


In [40]:
df_filtered = df_filtered.rename(columns={'query': 'before_gender_tansferred_query', 'gender_transferred_case_report': 'query'})

In [42]:
print(len(dict_list))

492


In [41]:
dict_list = df_filtered.to_dict(orient='records')
# print(dict_list)

with open('/home/gy237/project/llama3/total_final_test/change_gender_12_19/test.json', 'w', encoding='utf-8') as file:
    json.dump(dict_list, file, ensure_ascii=False, indent=4)

In [6]:
with open('/home/gy237/project/llama3/total_final_test/change_gender_12_19/test.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

In [8]:
print(data[0])
# {'id': 'JAMA_FINAL0', 'before_gender_tansferred_query': 'Develop a 10-Step differential diagnosis, for each step, give only one differential diagnosis with only the disease name at current step, do not provide repeated hypotheses or diagnoses that appear in previous steps. Rerank all 10 differential diagnosis using all patient information and test results. Let’s think step by step.\n### INPUT:\nA 77-year-old male with history of hypertension and asthma presented with 3 months of progressive decline in vision in the left eye. He had no known ocular history. At presentation, best-corrected visual acuity was 20/100 OD and 20/50 OS. Intraocular pressures were normal bilaterally and pupils were equally reactive. Examination of the right eye was unremarkable except for moderate cataract. The left-eye slitlamp examination was remarkable for a shallow but quiet anterior chamber and moderate cataract. Dilated fundus examination of the left eye revealed shallow peripheral serous choroidal detachments, bullous inferior macula-involving retinal detachment with shifting fluid, and diffuse hyperpigmented lesions involving the macula and superior fundus that corresponded to areas of nodular retinal pigment epithelium thickening on optical coherence tomography (Figure 1). These hyperpigmented lesions were associated with hyperautofluorescence and blockage on fluorescein and indocyanine green angiography. No retinal breaks were seen on scleral depressed examination. Ultrasound biomicroscopy demonstrated 360° ciliochoroidal effusion. Axial length was 24.30 mm OD and 24.32 mm OS. The patient denied a history of known refractive error. \n### OUTPUT:\n    ', 'answer': 'Idiopathic uveal effusion syndrome', 'topic': "['Retinal Disorders', 'Ophthalmic Imaging', 'Ophthalmology', 'Macular Diseases']", 'query': 'A 77-year-old female with history of hypertension and asthma presented with 3 months of progressive decline in vision in the left eye. She had no known ocular history. At presentation, best-corrected visual acuity was 20/100 OD and 20/50 OS. Intraocular pressures were normal bilaterally and pupils were equally reactive. Examination of the right eye was unremarkable except for moderate cataract. The left-eye slitlamp examination was remarkable for a shallow but quiet anterior chamber and moderate cataract. Dilated fundus examination of the left eye revealed shallow peripheral serous choroidal detachments, bullous inferior macula-involving retinal detachment with shifting fluid, and diffuse hyperpigmented lesions involving the macula and superior fundus that corresponded to areas of nodular retinal pigment epithelium thickening on optical coherence tomography (Figure 1). These hyperpigmented lesions were associated with hyperautofluorescence and blockage on fluorescein and indocyanine green angiography. No retinal breaks were seen on scleral depressed examination. Ultrasound biomicroscopy demonstrated 360° ciliochoroidal effusion. Axial length was 24.30 mm OD and 24.32 mm OS. The patient denied a history of known refractive error.'}

for i in data:
    i['query'] = f'''Develop a 10-Step differential diagnosis, for each step, give only one differential diagnosis with only the disease name at current step, do not provide repeated hypotheses or diagnoses that appear in previous steps. Rerank all 10 differential diagnosis using all patient information and test results. Let’s think step by step.\n### INPUT:\n{i['query']}\n### OUTPUT:\n    '''

{'id': 'JAMA_FINAL0', 'before_gender_tansferred_query': 'Develop a 10-Step differential diagnosis, for each step, give only one differential diagnosis with only the disease name at current step, do not provide repeated hypotheses or diagnoses that appear in previous steps. Rerank all 10 differential diagnosis using all patient information and test results. Let’s think step by step.\n### INPUT:\nA 77-year-old male with history of hypertension and asthma presented with 3 months of progressive decline in vision in the left eye. He had no known ocular history. At presentation, best-corrected visual acuity was 20/100 OD and 20/50 OS. Intraocular pressures were normal bilaterally and pupils were equally reactive. Examination of the right eye was unremarkable except for moderate cataract. The left-eye slitlamp examination was remarkable for a shallow but quiet anterior chamber and moderate cataract. Dilated fundus examination of the left eye revealed shallow peripheral serous choroidal detach

In [9]:
print(data[0])

{'id': 'JAMA_FINAL0', 'before_gender_tansferred_query': 'Develop a 10-Step differential diagnosis, for each step, give only one differential diagnosis with only the disease name at current step, do not provide repeated hypotheses or diagnoses that appear in previous steps. Rerank all 10 differential diagnosis using all patient information and test results. Let’s think step by step.\n### INPUT:\nA 77-year-old male with history of hypertension and asthma presented with 3 months of progressive decline in vision in the left eye. He had no known ocular history. At presentation, best-corrected visual acuity was 20/100 OD and 20/50 OS. Intraocular pressures were normal bilaterally and pupils were equally reactive. Examination of the right eye was unremarkable except for moderate cataract. The left-eye slitlamp examination was remarkable for a shallow but quiet anterior chamber and moderate cataract. Dilated fundus examination of the left eye revealed shallow peripheral serous choroidal detachments, bullous inferior macula-involving retinal detachment with shifting fluid, and diffuse hyperpigmented lesions involving the macula and superior fundus that corresponded to areas of nodular retinal pigment epithelium thickening on optical coherence tomography (Figure 1). These hyperpigmented lesions were associated with hyperautofluorescence and blockage on fluorescein and indocyanine green angiography. No retinal breaks were seen on scleral depressed examination. Ultrasound biomicroscopy demonstrated 360° ciliochoroidal effusion. Axial length was 24.30 mm OD and 24.32 mm OS. The patient denied a history of known refractive error. \n### OUTPUT:\n    ', 'answer': 'Idiopathic uveal effusion syndrome', 'topic': "['Retinal Disorders', 'Ophthalmic Imaging', 'Ophthalmology', 'Macular Diseases']", 'query': 'Develop a 10-Step differential diagnosis, for each step, give only one differential diagnosis with only the disease name at current step, do not provide repeated hypotheses or diagnoses that appear in previous steps. Rerank all 10 differential diagnosis using all patient information and test results. Let’s think step by step.\n### INPUT:\nA 77-year-old female with history of hypertension and asthma presented with 3 months of progressive decline in vision in the left eye. She had no known ocular history. At presentation, best-corrected visual acuity was 20/100 OD and 20/50 OS. Intraocular pressures were normal bilaterally and pupils were equally reactive. Examination of the right eye was unremarkable except for moderate cataract. The left-eye slitlamp examination was remarkable for a shallow but quiet anterior chamber and moderate cataract. Dilated fundus examination of the left eye revealed shallow peripheral serous choroidal detachments, bullous inferior macula-involving retinal detachment with shifting fluid, and diffuse hyperpigmented lesions involving the macula and superior fundus that corresponded to areas of nodular retinal pigment epithelium thickening on optical coherence tomography (Figure 1). These hyperpigmented lesions were associated with hyperautofluorescence and blockage on fluorescein and indocyanine green angiography. No retinal breaks were seen on scleral depressed examination. Ultrasound biomicroscopy demonstrated 360° ciliochoroidal effusion. Axial length was 24.30 mm OD and 24.32 mm OS. The patient denied a history of known refractive error.\n### OUTPUT:\n    '}

{'id': 'JAMA_FINAL0', 'before_gender_tansferred_query': 'Develop a 10-Step differential diagnosis, for each step, give only one differential diagnosis with only the disease name at current step, do not provide repeated hypotheses or diagnoses that appear in previous steps. Rerank all 10 differential diagnosis using all patient information and test results. Let’s think step by step.\n### INPUT:\nA 77-year-old male with history of hypertension and asthma presented with 3 months of progressive decline in vision in the left eye. He had no known ocular history. At presentation, best-corrected visual acuity was 20/100 OD and 20/50 OS. Intraocular pressures were normal bilaterally and pupils were equally reactive. Examination of the right eye was unremarkable except for moderate cataract. The left-eye slitlamp examination was remarkable for a shallow but quiet anterior chamber and moderate cataract. Dilated fundus examination of the left eye revealed shallow peripheral serous choroidal detach

In [11]:
train_dataset = Dataset.from_list(data)
t = Dataset.from_list(data[:10])

dataset_dict = DatasetDict({
    "train": train_dataset,
    "valid":t,
    "test": t
})
dataset_dict.push_to_hub("YBXL/JAMA_FINAL_test_gender_transferred")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/625 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/YBXL/JAMA_FINAL_test_gender_transferred/commit/a1fb59acfbb6ca0a410968ebc096fb167a5d4c52', commit_message='Upload dataset', commit_description='', oid='a1fb59acfbb6ca0a410968ebc096fb167a5d4c52', pr_url=None, pr_revision=None, pr_num=None)

In [46]:
print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['id', 'before_gender_tansferred_query', 'answer', 'topic', 'query'],
        num_rows: 492
    })
    valid: Dataset({
        features: ['id', 'before_gender_tansferred_query', 'answer', 'topic', 'query'],
        num_rows: 10
    })
    test: Dataset({
        features: ['id', 'before_gender_tansferred_query', 'answer', 'topic', 'query'],
        num_rows: 10
    })
})
